In [1]:
# 載入模組
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


url = 'https://mops.twse.com.tw/mops/web/ajax_t05st03'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}

In [2]:
# 上市櫃股票代號
stock = pd.read_csv('stock.csv', encoding= 'utf8')
stockno = stock['股票代號']

In [3]:
for idx, i in stockno[0:5].items():
    # 等待1.5秒
    time.sleep(1.5)
    payload = {
        'encodeURIComponent':'1',
        'step':'1',
        'firstin':'1',
        'off':'1',
        'queryName':'co_id',
        'inpuType':'co_id',
        'TYPEK':'all',
        'co_id': i,
        }

    # 取得網頁
    resp = requests.post(url, data=payload,headers = headers)

    # 解析網頁
    soup = BeautifulSoup(resp.text, 'html.parser')

    # 取得全部有效table
    table = soup.find_all('table',{'class': 'hasBorder'})

    # 第一個table

    # 取得資料
    data = [th.text.replace('\n', '') for th in table[0].find('tr').find_all('td')]
    # 清理資料
    out = ["".join(i.split()) for i in data ]
    # 處理欄位
    head = [th.text.replace('\n', '') for th in table[0].find('tr').find_all('th') if th.text != "本公司"]

    # 第二個table，有資料沒用tr包住，故

    # 取得資料
    data2 = [th.text.replace('\n', '') for th in table[1].find_all('td')]
    # 清理資料
    out2 = ["".join(i.split()) for i in data2 ]
    # 處理欄位
    head2 = [th.text.replace('\n', '') for th in table[1].find_all('th')  if "本公司" not in th.text]

    # 合併兩段table資料與欄位
    columns = head + head2
    rowdata = out + out2

    # list在dataframe預設為column資料，所以傳入時，用[]強制為行資料
    if idx ==0:
        df = pd.DataFrame([rowdata],columns=columns)
    else:
        df.loc[idx] = rowdata

# 存檔
df.to_csv('.\\files\\all_06.csv',index=False)

In [4]:
df

,股票代號,產業類別,外國企業註冊地國,公司名稱,總機,地址,董事長,總經理,發言人,發言人職稱,...,投資人關係聯絡電話,投資人關係電子郵件,公司網站內利害關係人專區網址,變更前名稱,變更前簡稱,公司名稱變更核准日期,月制會計年度（空白表曆年制）,之前採,月制會計年度,編製財務報告類型
0,1101,水泥工業,－,台灣水泥股份有限公司,(02)2531-7099,台北市中山北路2段113號,張安平,李鐘培,黃健強,資深副總經理,...,02-25317099分機20358,ir@taiwancement.com,http://www.taiwancement.com/tw/csr/csr5-1.html,,,0,,,,●合併○個別
1,1102,水泥工業,－,亞洲水泥股份有限公司,02-2733-8000,台北市大安區敦化南路2段207號30、31樓,徐旭東,李坤炎,周維崑,副總經理,...,0227338000ext.8336,ir@acc.com.tw,http://www.acc.com.tw/tw/stakeholder/stakehold...,,,0,,,,●合併○個別
2,1103,水泥工業,－,嘉新水泥股份有限公司,(02)2551-5211,台北市中山北路2段96號,張剛綸,祁士鉅,王立心,執行副總經理,...,(02)2551-5211#243,ir@chcgroup.com.tw,www.chcgroup.com.tw/index.php?route=system/sys...,,,0,,,,●合併○個別
3,1104,水泥工業,－,環球水泥股份有限公司,02-25077801,台北市南京東路二段125號10樓,侯博義,侯智升,楊宗仁,副總經理,...,02-2507-7801,p1590@ucctw.com,http://www.ucctw.com/ucctw/index.php?option=co...,,,0,,,,●合併○個別
4,1108,水泥工業,－,幸福水泥股份有限公司,25092188,台北市松江路237號15樓,陳兩傳,陳兩傳,溫秀菊,財務部經理,...,25092188,wendy@luckygrp.com.tw,http://www.luckygrp.com.tw/tw/index.asp?au_id=...,,,0,,,,●合併○個別
